In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Подготовка данных

Загрузка данных

In [3]:
xlsx_file = pd.ExcelFile('input/БОГРАД_PISY.xlsx')
trees = ['pisy_01a', 'pisy_01b', 'pisy_02a', 'pisy_03a', 'pisy_07a', 'pisy_12b', 'pisy_14a']
columns = {_:f'D{__}' if __<16 else f'CWT{__-15}' for _, __ in zip(range(2,32), range(1,31))}
columns[0] = 'Tree'
columns[1] = 'Year'

Функция нормализации трахеид

In [4]:
def get_normalized_list(x: list, norm : int):
    """
    Функция получения нормированного списка
    :param x: список для нормирования
    :param norm: норма
    :return: l_norm - нормированный к e список l
    """
    l_raw = []  # промежуточный список
    n = len(x)
    for i in range(n):
        for j in range(norm):
            l_raw += [x[i]]
    l_norm = []
    for i in range(norm):
        l_norm += [1 / n * sum([l_raw[j] for j in range(n * i, n * (i + 1))])]
    return l_norm

Нормализуем входные данные

In [5]:
dataframes = []
for tree in trees:
    df = xlsx_file.parse(tree)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.dropna(axis=0)

    norm_traches = dict()
    for year in set(df['Год']):
        norm_traches[int(year)] = [tree, int(year)] + get_normalized_list(list(df[df['Год']==year]['Dmean']), 15)+ get_normalized_list(list(df[df['Год']==year]['CWTmean']), 15)
    
    dataframes += [pd.DataFrame(norm_traches).transpose().rename(columns=columns).reset_index(drop=True)]

df = pd.concat(dataframes).reset_index(drop=True)

Сохраняем их в .xlsx файл

In [6]:
df.to_excel('output/Bograd_PISY_normalized.xlsx', index=False)

Рассчёт средних значений трахеид по годам и по деревьям:

In [7]:
mean_objects_years = dict()

for year in set(df['Year']):
    temp_data = df[df['Year']==year]
    if len(temp_data) > 3:
        mean_objects_years[year] = temp_data.mean()[1:]


mean_objects_trees = dict()

for tree in set(df['Tree']):
    mean_objects_trees[tree] = df[df['Tree']==tree].mean()[1:]

Считаем средние значения трахеид по всем записям

In [8]:
global_mean = df.mean()[1:]

Строим таблицы объектов для метода A -- кластеризация отклонений средних объектов по году от среднего глобального объекта:

In [9]:
quotient_deviation_df_A = []
difference_deviation_df_A = []

_columns = {_:f'D{_}' if _<16 else f'CWT{_-15}' for _ in  range(1,31)}
_columns[0] = 'Year'

for year, mean_obj in mean_objects_years.items():
    quotient_deviation_df_A += [[year] + list(mean_obj/global_mean)]
    difference_deviation_df_A += [[year] + list(mean_obj-global_mean)]

quotient_deviation_df_A = pd.DataFrame(quotient_deviation_df_A).rename(columns=_columns)
difference_deviation_df_A = pd.DataFrame(difference_deviation_df_A).rename(columns=_columns)

In [10]:
quotient_deviation_df_A.to_excel('output/quotient_deviation_df_A.xlsx', index=False)
difference_deviation_df_A.to_excel('output/difference_deviation_df_A.xlsx', index=False)

Строим таблицы объектов для метода B -- кластеризация средних отклонений объектов по году от среднего объекта по дереву:

In [11]:
qd_df_B = []
dd_df_B = []

quotient_deviation_df_B = dict()
difference_deviation_df_B = dict()


for _, row in df.iterrows():
    qd_df_B += [[row[0], row[1]] + list(row[2:] / mean_objects_trees[row[0]])]
    dd_df_B +=  [[row[0], row[1]] + list(row[2:] - mean_objects_trees[row[0]])]

qd_df_B = pd.DataFrame(qd_df_B).rename(columns=columns)
dd_df_B = pd.DataFrame(dd_df_B).rename(columns=columns)

for year in set(df['Year']):
    temp_data_q = qd_df_B[qd_df_B['Year']==year]
    temp_data_d = dd_df_B[dd_df_B['Year']==year]
    if len(temp_data_q) > 3:
        quotient_deviation_df_B[year] = temp_data_q.mean()[1:]
        difference_deviation_df_B[year] = temp_data_d.mean()[1:]

quotient_deviation_df_B = pd.DataFrame(quotient_deviation_df_B).transpose()
difference_deviation_df_B = pd.DataFrame(difference_deviation_df_B).transpose()

In [12]:
quotient_deviation_df_B.to_excel('output/quotient_deviation_df_B.xlsx', index=True)
difference_deviation_df_B.to_excel('output/difference_deviation_df_B.xlsx', index=True)

# Кластеризация

In [13]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


def get_models(data, name='Data'):
    models = []
    inertia = []
    silhouette = []
    print(name, end=': ')
    for n_clusters in range(2, 16):
        print(n_clusters, end=', ')
        # Описываем модель
        model = KMeans(n_clusters=n_clusters, max_iter=5000, random_state=0)

        # Проводим моделирование
        model.fit(data)

        # Предсказание на всем наборе данных
        all_predictions = model.predict(data)

        # Распихиваем точки по кластерам
        clusters = [[] for i in range(n_clusters)]
        for i, num in enumerate(all_predictions):
            clusters[num] += [data[i]]
        
        models += [model]
        inertia += [model.inertia_]
        silhouette += [silhouette_score(data, model.labels_, metric='euclidean')]
    print('done!')
    return models, inertia, silhouette

Тут формируем массивы обучабщих выборок для обоих методов:

In [14]:
quo_data_A  = np.array(quotient_deviation_df_A.drop(['Year'], axis=1))
diff_data_A = np.array(difference_deviation_df_A.drop(['Year'], axis=1))
quo_data_B  = np.array(quotient_deviation_df_B)
diff_data_B = np.array(difference_deviation_df_B)

Обучаем модели для каждого метода:

In [15]:
models_quo_A, i_1, s_1 = get_models(quo_data_A, 'A quo')
models_diff_A, i_2, s_2 = get_models(diff_data_A, 'A diff')
models_quo_B, i_1, s_1 = get_models(quo_data_B, 'B quo')
models_diff_B, i_2, s_2 = get_models(diff_data_B, 'B diff')

A quo: 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, done!
A diff: 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, done!
B quo: 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, done!
B diff: 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, done!


Функция сохранения графиков кластеров и их средних объектов:

In [16]:
def save_clusters(models, data, amount=15, num=3, data_type='below', ylim=None):
    all_predictions = models[num-2].predict(data)
    try:
        os.mkdir(f'output/{data_type}/')
    except Exception as e:
        pass
    try:
        os.mkdir(f'output/{data_type}/{num}_clust/')
    except Exception as e:
        pass
    for j in range(num):
        fig, ax = plt.subplots( nrows=1, ncols=1)
        ax.plot(range(1,amount+1), models[num-2].cluster_centers_[j])
        ax.set_title(f'Mean object (cluster №{j+1})')
        if ylim:
            ax.set_ylim(ylim)
        fig.savefig(f'output/{data_type}/{num}_clust/mean_obj_cluster_{j+1}.png')
        plt.close(fig)
        
        fig, ax = plt.subplots( nrows=1, ncols=1)
        els = 0
        for i in range(len(all_predictions)):
            if all_predictions[i] == j:
                ax.plot(range(1,amount+1), data[i], label=str(i))
                els += 1
        #legend(frameon=False)
        if ylim:
            ax.set_ylim(ylim)
        ax.set_title(f'Cluster №{j+1} ({els} elements)')
        fig.savefig(f'output/{data_type}/{num}_clust/cluster_{j+1}.png') 
        plt.close(fig)

Сохраняем графики для всех моделей:

In [17]:
for _ in range(3, 6):
    save_clusters(models_quo_A, quo_data_A, 30, _, 'A_Quotient', [0.5, 1.5])
    save_clusters(models_diff_A, diff_data_A, 30, _, 'A_Difference', ylim=[-15, 15])
    save_clusters(models_quo_B, quo_data_B, 30, _, 'B_Quotient', [0.5, 1.5])
    save_clusters(models_diff_B, diff_data_B, 30, _, 'B_Difference', ylim=[-15, 15])

Сохраняем таблицы кластеризованных объектов:

In [18]:
for i in range(2, 16):
    quotient_deviation_df_A[f'Class {i}'] = models_quo_A[i-2].predict(quo_data_A)
    difference_deviation_df_A[f'Class {i}'] = models_diff_A[i-2].predict(diff_data_A)
    quotient_deviation_df_B[f'Class {i}'] = models_quo_B[i-2].predict(quo_data_B)
    difference_deviation_df_B[f'Class {i}'] = models_diff_B[i-3].predict(diff_data_B)

quotient_deviation_df_A.to_excel('output/quotient_deviation_df_A_CLASSIFIED.xlsx', index=False)
difference_deviation_df_A.to_excel('output/difference_deviation_df_A_CLASSIFIED.xlsx', index=False)
quotient_deviation_df_B.to_excel('output/quotient_deviation_df_B_CLASSIFIED.xlsx', index=True)
difference_deviation_df_B.to_excel('output/difference_deviation_df_B_CLASSIFIED.xlsx', index=True)

# Сравнение методов:

Строим таблицу коэффициентов корреляции Спирмена для двух методов:

In [19]:
QCorr = quotient_deviation_df_A.drop(['Year', 'Class 3', 'Class 4', 'Class 5'], axis=1).corrwith(
        quotient_deviation_df_B.drop(['Class 3', 'Class 4', 'Class 5'], axis=1).reset_index(drop=True),
        method='spearman')

DCorr = difference_deviation_df_A.drop(['Year', 'Class 3', 'Class 4', 'Class 5'], axis=1).corrwith(
        difference_deviation_df_B.drop(['Class 3', 'Class 4', 'Class 5'], axis=1).reset_index(drop=True),
        method='spearman')

In [20]:
spearman_corr_df = pd.DataFrame({'Quotient': QCorr, 'Difference':DCorr})
spearman_corr_df.to_excel('output/spearman_correlation.xlsx')

Строим сравнительные графики для двух методов и двух типов отклонений:

In [21]:
d_names = [
    ['D1', 'D2', 'D3'],
    ['D4', 'D5', 'D6'],
    ['D7', 'D8', 'D9'],
    ['D10', 'D11', 'D12'],
    ['D13', 'D14', 'D15']
]

cwt_names = [
    ['CWT1', 'CWT2', 'CWT3'],
    ['CWT4', 'CWT5', 'CWT6'],
    ['CWT7', 'CWT8', 'CWT9'],
    ['CWT10', 'CWT11', 'CWT12'],
    ['CWT13', 'CWT14', 'CWT15']
]

def save_corr_plots(df1, df2, names, corr_row, output_name=''):
    fig, ax = plt.subplots(5,3, figsize=(15,12), dpi=300)
    for i, row in enumerate(names):
        for j, el in enumerate(row):
            ax[i, j].scatter(df1[el], df2[el])
            ax[i, j].text(0.1, 0.8, f"{el},  r={corr_row[el]:0.4f}", transform=ax[i, j].transAxes)
    fig.savefig(f'output/{output_name}.png', dpi=300) 
    plt.close(fig)

save_corr_plots(quotient_deviation_df_A, quotient_deviation_df_B, d_names, spearman_corr_df['Quotient'], 'Quotient_D_corr')
save_corr_plots(quotient_deviation_df_A, quotient_deviation_df_B, cwt_names, spearman_corr_df['Quotient'], 'Quotient_CWT_corr')
save_corr_plots(difference_deviation_df_A, difference_deviation_df_B, d_names, spearman_corr_df['Difference'], 'Difference_D_corr')
save_corr_plots(difference_deviation_df_A, difference_deviation_df_B, cwt_names, spearman_corr_df['Difference'], 'Difference_CWT_corr')